In [ ]:
# !pip install openpyxl

In [ ]:
import numpy as np
import pandas as pd


file_path = '/content/FInal_data_20_columns.xlsx'
df = pd.read_excel(file_path)

df = df.astype(str)
# df = df.fillna("")
df.head(2)

In [ ]:
df = df.iloc[:2000]

In [ ]:
# Function to check and format text
def check_text(text):
    if text in ['Not defined']:
        return 'NaN'
    return text

# Create the paragraph column
for col in df.columns:
  df[col] = df[col].apply(lambda txt: check_text(txt))



In [ ]:
df.shape

In [ ]:
df.head(2)

In [ ]:
def lower_text(text):
    return text.lower()

df = df.applymap(lambda x: lower_text(x))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
text_data = df['Paragraph']
target_data = df.drop(columns=['Paragraph'])

In [ ]:
print(text_data.shape)
print(target_data.shape)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, TimeDistributed, Bidirectional, Concatenate, GRU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
print(max([len(sen.split()) for sen in text_data]))

In [ ]:
max_words_in_sentense = max([len(sen.split()) for sen in text_data])
tokenizer = Tokenizer(num_words=max_words_in_sentense)
tokenizer.fit_on_texts(text_data)
# word_index = tokenizer.word_index

In [ ]:
# special_tokens = ['<start>', '<end>']
# for token in special_tokens:
#     tokenizer.word_index[token] = len(tokenizer.word_index) + 1


tokenizer.word_index['<start>'] = 0
tokenizer.index_word[0] = '<start>'

word_index = tokenizer.word_index

In [ ]:
start_token_id = word_index['<start>']
# end_token_id = word_index['<end>']

In [ ]:
# word_index

In [ ]:
# text_data[0]

In [ ]:
tokenizer.index_word = {index: word for word, index in word_index.items()}
sequences = tokenizer.texts_to_sequences(text_data)
# max_sequence_length = 700
max_sequence_length = max_words_in_sentense
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')

In [ ]:
padded_sequences.shape

In [ ]:
vocab_size_input = len(tokenizer.word_counts)
print(vocab_size_input)

In [ ]:
len(tokenizer.word_counts)

In [ ]:
# padded_sequences[10]

In [ ]:
def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

glove_file_path = '/content/drive/MyDrive/glove.6B.50d.txt'
embeddings_index = load_glove_embeddings(glove_file_path)
print(f'Loaded {len(embeddings_index)} word vectors.')

In [ ]:
embedding_dim = 50

embedding_matrix_input = np.zeros((vocab_size_input, embedding_dim))
for word, i in word_index.items():
    if i < vocab_size_input:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix_input[i] = embedding_vector

In [ ]:
# embedding_matrix[40]

In [ ]:
# Separate tokenizer for target data
tokenizer_target = Tokenizer(num_words= max_words_in_sentense)
tokenizer_target.fit_on_texts(target_data.values.flatten())
word_index_target = tokenizer_target.word_index

In [ ]:
latent_dim = 32
x_max_sequence_length = max_words_in_sentense
# y_max_sequence_length = max_words_in_sentense
y_max_sequence_length = 70

In [ ]:
# target_sequences = np.zeros((len(df), target_data.shape[1], max_sequence_length), dtype='int32')

# for i, (key, sentences) in enumerate(target_data.items()):
#     tokenizer.fit_on_texts(sentences)
#     sequences_y = tokenizer.texts_to_sequences(sentences)
#     padded_sequences_y = pad_sequences(sequences_y, maxlen=max_sequence_length, padding='post')
#     target_sequences[:, i, :] = padded_sequences_y


target_sequences = np.zeros((len(df), target_data.shape[1], y_max_sequence_length), dtype='int32')
for i, (key, sentences) in enumerate(target_data.items()):
    sequences_target = tokenizer_target.texts_to_sequences(sentences)
    padded_sequences_target = pad_sequences(sequences_target, maxlen=y_max_sequence_length, padding='post')
    target_sequences[:, i, :] = padded_sequences_target

In [ ]:
# Assuming your model and data preparation are set up as previously discussed

# Splitting data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, target_sequences, test_size=0.2, random_state=42)


In [ ]:
y_vocab_size_input = len(tokenizer.word_counts)

In [ ]:
def add_start_token(sequences, start_token_id):
    return np.array([[start_token_id] + seq for seq in sequences])

y_train = add_start_token(y_train, start_token_id)


In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, TimeDistributed, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2


        ## model 1
# # Encoder
# encoder_inputs = Input(shape=(max_sequence_length,), name='encoder_inputs')
# encoder_embedding = Embedding(input_dim=max_words, output_dim=embedding_dim, weights=[embedding_matrix],
#                               input_length=max_sequence_length, trainable=False)(encoder_inputs)
# encoder_lstm = LSTM(latent_dim, return_state=True, name='encoder_lstm')
# encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
# encoder_states = [state_h, state_c]

# # Decoder
# decoder_inputs = Input(shape=(12, max_sequence_length,), name='decoder_inputs')
# decoder_embedding = Embedding(input_dim=max_words, output_dim=embedding_dim, trainable=False)(decoder_inputs)

# decoder_outputs = []
# for i in range(target_data.shape[1]):
#     decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=False, name=f'decoder_lstm_{i}')
#     decoder_lstm_output = decoder_lstm(decoder_embedding[:, i, :], initial_state=encoder_states)
#     decoder_dense = TimeDistributed(Dense(max_words, activation='softmax'), name=f'decoder_dense_{i}')
#     decoder_outputs.append(decoder_dense(decoder_lstm_output))




            ## model 2
# # Encoder
# encoder_inputs = Input(shape=(max_sequence_length,), name='encoder_inputs')
# encoder_embedding = Embedding(input_dim=max_words, output_dim=embedding_dim, weights=[embedding_matrix],
#                               input_length=max_sequence_length, trainable=False)(encoder_inputs)
# encoder_lstm = LSTM(latent_dim, return_state=True, name='encoder_lstm')
# encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
# encoder_states = [state_h, state_c]

# # Decoder
# # decoder_inputs = Input(shape=(12, max_sequence_length,), name='decoder_inputs')
# decoder_inputs = Input(shape=(12, None,), name='decoder_inputs')
# decoder_embedding = TimeDistributed(Embedding(input_dim=max_words, output_dim=embedding_dim, weights=[embedding_matrix],
#                                               trainable=False))(decoder_inputs)

# # Process each target sequence individually
# decoder_outputs = []
# for i in range(12):
#     decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=False, name=f'decoder_lstm_{i}')
#     decoder_lstm_output = decoder_lstm(decoder_embedding[:, i, :], initial_state=encoder_states)
#     decoder_dense = TimeDistributed(Dense(max_words, activation='softmax'), name=f'decoder_dense_{i}')
#     decoder_outputs.append(decoder_dense(decoder_lstm_output))




    ### model with bidirectional layer .. good accuracy
# from keras.layers import Input, Embedding, LSTM, Bidirectional, Dense, TimeDistributed, Lambda, Concatenate
# from keras.models import Model
# import keras.backend as K

# # Encoder
# encoder_inputs = Input(shape=(max_sequence_length,), name='encoder_inputs')
# encoder_embedding = Embedding(input_dim=vocab_size_input, output_dim=embedding_dim, weights=[embedding_matrix_input],
#                               input_length=max_sequence_length, trainable=False)(encoder_inputs)
# encoder_lstm = Bidirectional(LSTM(latent_dim, return_state=True, name='encoder_lstm', dropout=0.5, recurrent_dropout=0.5))
# encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_lstm(encoder_embedding)

# # Concatenate forward and backward states
# state_h = Concatenate()([forward_h, backward_h])
# state_c = Concatenate()([forward_c, backward_c])

# # Decoder
# decoder_inputs = Input(shape=(31, y_max_sequence_length,), name='decoder_inputs')
# decoder_embedding = TimeDistributed(Embedding(input_dim=y_vocab_size_input, output_dim=embedding_dim, weights=[embedding_matrix_input],
#                                               trainable=False))(decoder_inputs)

# decoder_outputs = []
# for i in range(31):
#     decoder_lstm = LSTM(latent_dim * 2, return_sequences=True, return_state=False, name=f'decoder_lstm_{i}',
#                         dropout=0.5, recurrent_dropout=0.5)
#     decoder_lstm_output = decoder_lstm(decoder_embedding[:, i, :], initial_state=[state_h, state_c])
#     decoder_dense = TimeDistributed(Dense(y_vocab_size_input, activation='softmax'), name=f'decoder_dense_{i}')
#     decoder_outputs.append(decoder_dense(decoder_lstm_output))

# model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# model.summary()



#           ### multiple Bidirectional and LSTM
# # Encoder
# encoder_inputs = Input(shape=(max_sequence_length,), name='encoder_inputs')
# encoder_embedding = Embedding(input_dim=vocab_size_input, output_dim=embedding_dim, weights=[embedding_matrix_input],
#                               input_length=max_sequence_length, trainable=False)(encoder_inputs)
# encoder_lstm1 = Bidirectional(LSTM(latent_dim, return_sequences=True, name='encoder_lstm_1', dropout=0.5, recurrent_dropout=0.5))(encoder_embedding)
# encoder_lstm2, forward_h2, forward_c2, backward_h2, backward_c2 = Bidirectional(LSTM(latent_dim, return_sequences=True, return_state=True,
#                                                         name='encoder_lstm_2', dropout=0.5, recurrent_dropout=0.5))(encoder_lstm1)
# state_h = Concatenate()([forward_h2, backward_h2])
# state_c = Concatenate()([forward_c2, backward_c2])
# # Decoder
# decoder_inputs = Input(shape=(31, y_max_sequence_length,), name='decoder_inputs')
# decoder_embedding = TimeDistributed(Embedding(input_dim=y_vocab_size_input, output_dim=embedding_dim, weights=[embedding_matrix_input],
#                                               trainable=False))(decoder_inputs)
# decoder_outputs = []
# for i in range(31):
#     decoder_lstm_1 = LSTM(latent_dim * 2, return_sequences=True, return_state=False, name=f'decoder_lstm_1_{i}',
#                         dropout=0.5, recurrent_dropout=0.5)
#     decoder_lstm_2 = LSTM(latent_dim * 2, return_sequences=True, return_state=False, name=f'decoder_lstm_2_{i}',
#                         dropout=0.5, recurrent_dropout=0.5)
#     decoder_lstm_output_1 = decoder_lstm_1(decoder_embedding[:, i, :], initial_state=[state_h, state_c])
#     decoder_lstm_output_2 = decoder_lstm_2(decoder_lstm_output_1, initial_state=[state_h, state_c])
#     decoder_dense = TimeDistributed(Dense(y_vocab_size_input, activation='softmax'), name=f'decoder_dense_{i}')
#     decoder_outputs.append(decoder_dense(decoder_lstm_output_2))
# model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# model.summary()





              #### multiple Bidirectional and LSTM and regularization
# # Encoder
# encoder_inputs = Input(shape=(max_sequence_length,), name='encoder_inputs')
# encoder_embedding = Embedding(input_dim=vocab_size_input, output_dim=embedding_dim, weights=[embedding_matrix_input],
#                               input_length=max_sequence_length, trainable=False,
#                               embeddings_regularizer=l2(1e-4))(encoder_inputs)
# encoder_lstm1 = Bidirectional(LSTM(latent_dim, return_sequences=True, name='encoder_lstm_1', dropout=0.5, recurrent_dropout=0.5,
#                                    kernel_regularizer=l2(1e-4), recurrent_regularizer=l2(1e-4)))(encoder_embedding)
# encoder_lstm2, forward_h2, forward_c2, backward_h2, backward_c2 = Bidirectional(LSTM(latent_dim, return_sequences=True, return_state=True,
#                                                         name='encoder_lstm_2', dropout=0.5, recurrent_dropout=0.5,
#                                                         kernel_regularizer=l2(1e-4), recurrent_regularizer=l2(1e-4)))(encoder_lstm1)
# state_h = Concatenate()([forward_h2, backward_h2])
# state_c = Concatenate()([forward_c2, backward_c2])
# # Decoder
# decoder_inputs = Input(shape=(31, y_max_sequence_length,), name='decoder_inputs')
# decoder_embedding = TimeDistributed(Embedding(input_dim=y_vocab_size_input, output_dim=embedding_dim, weights=[embedding_matrix_input],
#                                               trainable=False, embeddings_regularizer=l2(1e-4)))(decoder_inputs)
# decoder_outputs = []
# for i in range(31):
#     decoder_lstm_1 = LSTM(latent_dim * 2, return_sequences=True, return_state=False, name=f'decoder_lstm_1_{i}',
#                           dropout=0.5, recurrent_dropout=0.5, kernel_regularizer=l2(1e-4), recurrent_regularizer=l2(1e-4))
#     decoder_lstm_2 = LSTM(latent_dim * 2, return_sequences=True, return_state=False, name=f'decoder_lstm_2_{i}',
#                           dropout=0.5, recurrent_dropout=0.5, kernel_regularizer=l2(1e-4), recurrent_regularizer=l2(1e-4))
#     decoder_lstm_output_1 = decoder_lstm_1(decoder_embedding[:, i, :], initial_state=[state_h, state_c])
#     decoder_lstm_output_2 = decoder_lstm_2(decoder_lstm_output_1, initial_state=[state_h, state_c])
#     decoder_dense = TimeDistributed(Dense(y_vocab_size_input, activation='softmax', kernel_regularizer=l2(1e-4)), name=f'decoder_dense_{i}')
#     decoder_outputs.append(decoder_dense(decoder_lstm_output_2))
# model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# model.summary()





      ### GRU model
# from keras.layers import Input, Embedding, LSTM, Bidirectional, Dense, TimeDistributed, Lambda, Concatenate, GRU
# from keras.models import Model
# import keras.backend as K

# # Encoder
# encoder_inputs = Input(shape=(max_sequence_length,), name='encoder_inputs')
# encoder_embedding = Embedding(input_dim=vocab_size_input, output_dim=embedding_dim, weights=[embedding_matrix_input],
#                               input_length=max_sequence_length, trainable=False)(encoder_inputs)
# encoder_gru = Bidirectional(GRU(latent_dim, return_state=True, name='encoder_gru', dropout=0.5, recurrent_dropout=0.5))
# encoder_outputs, forward_h, backward_h = encoder_gru(encoder_embedding)

# # Concatenate forward and backward states
# state_h = Concatenate()([forward_h, backward_h])

# # Decoder
# decoder_inputs = Input(shape=(31, y_max_sequence_length,), name='decoder_inputs')  # +1 for the start token
# decoder_embedding = TimeDistributed(Embedding(input_dim=vocab_size_input, output_dim=embedding_dim, weights=[embedding_matrix_input],
#                                               trainable=False))(decoder_inputs)

# decoder_outputs = []
# for i in range(31):
#     decoder_gru = GRU(latent_dim * 2, return_sequences=True, return_state=False, name=f'decoder_gru_{i}', dropout=0.5, recurrent_dropout=0.5)
#     decoder_gru_output = decoder_gru(decoder_embedding[:, i, :], initial_state=[state_h])
#     decoder_dense = TimeDistributed(Dense(vocab_size_input, activation='softmax'), name=f'decoder_dense_{i}')
#     decoder_outputs.append(decoder_dense(decoder_gru_output))

# model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# model.summary()




            #### model 5
# from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization
### Encoder
# encoder_inputs = Input(shape=(max_sequence_length,), name='encoder_inputs')
# encoder_embedding = Embedding(input_dim=vocab_size_input, output_dim=embedding_dim, weights=[embedding_matrix_input],
#                               input_length=max_sequence_length, trainable=False)(encoder_inputs)
# transformer_layer = MultiHeadAttention(num_heads=4, key_dim=latent_dim)
# transformer_output = transformer_layer(encoder_embedding, encoder_embedding, encoder_embedding)
# transformer_output = LayerNormalization()(transformer_output + encoder_embedding)
#### Decoder
# decoder_inputs = Input(shape=(31, y_max_sequence_length,), name='decoder_inputs')
# decoder_embedding = TimeDistributed(Embedding(input_dim=vocab_size_input, output_dim=embedding_dim, weights=[embedding_matrix_input],
#                                               trainable=False))(decoder_inputs)
# decoder_outputs = []
# for i in range(31):
#     decoder_transformer_output = transformer_layer(decoder_embedding[:, i, :], transformer_output, transformer_output)
#     decoder_transformer_output = LayerNormalization()(decoder_transformer_output + decoder_embedding[:, i, :])
#     decoder_dense = TimeDistributed(Dense(vocab_size_input, activation='softmax'), name=f'decoder_dense_{i}')
#     decoder_outputs.append(decoder_dense(decoder_transformer_output))
# model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# model.summary()

In [ ]:
# print("encoder input shape  => ", encoder_inputs.shape)
# print("encoder embedding shape  => ", encoder_embedding.shape)
# # print("encoder shape  => ", encoder_outputs.shape, forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)

# print("encoder shape  => ", forward_h2.shape, forward_c2.shape, backward_h2.shape, backward_c2.shape)
# print("encoder output shape  => ", encoder_lstm2.shape)

In [ ]:
# print("state_h  ", state_h.shape)
# print("state_c  ", state_c.shape)

In [ ]:
# print("decoder input shape  => ", decoder_inputs.shape)
# print("decoder embedding shape  => ", decoder_embedding.shape)
# print("decoder lstm out shape  => ", decoder_lstm_output.shape)
# print("decoder dense  => ", decoder_dense)

In [ ]:
# print("decoder dense  => ", decoder_dense.input_shape)

In [ ]:
# from tensorflow.keras.utils import plot_model
# plot_model(model)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
!pip install keras-tuner

In [ ]:
# from keras_tuner import HyperModel
# import keras_tuner as kt
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.regularizers import l1_l2, l2

# class NLPHyperModel(HyperModel):
#     def __init__(self, vocab_size_input, embedding_dim, max_sequence_length, embedding_matrix_input, y_vocab_size_input, y_max_sequence_length):
#         self.vocab_size_input = vocab_size_input
#         self.embedding_dim = embedding_dim
#         self.max_sequence_length = max_sequence_length
#         self.embedding_matrix_input = embedding_matrix_input
#         self.y_vocab_size_input = y_vocab_size_input
#         self.y_max_sequence_length = y_max_sequence_length

#     def build(self, hp):
#         latent_dim = hp.Int('latent_dim', min_value=32, max_value=256, step=32)
#         dropout_rate = hp.Float('dropout_rate', min_value=0.3, max_value=0.7, step=0.1)
#         recurrent_dropout_rate = hp.Float('recurrent_dropout_rate', min_value=0.3, max_value=0.7, step=0.1)
#         learning_rate = hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='LOG')
#         # learning_rate = hp.Float('learning_rate', min_value=0.001, max_value=0.01, sampling='LOG')
#         l1_reg = hp.Float('l1_reg', min_value=1e-5, max_value=1e-2, sampling='LOG')
#         l2_reg = hp.Float('l2_reg', min_value=1e-5, max_value=1e-2, sampling='LOG')
#         ### l1_l2(l1=l1_reg, l2=l2_reg)
#         ### l2(1e-4)


#         # Encoder
#         encoder_inputs = Input(shape=(self.max_sequence_length,), name='encoder_inputs')
#         encoder_embedding = Embedding(input_dim=self.vocab_size_input, output_dim=self.embedding_dim, weights=[self.embedding_matrix_input],
#                                       input_length=self.max_sequence_length, trainable=False)(encoder_inputs)
#         encoder_lstm1 = Bidirectional(LSTM(latent_dim, return_sequences=True, name='encoder_lstm_1', dropout=dropout_rate,
#                                            recurrent_dropout=recurrent_dropout_rate, kernel_regularizer=l2(1e-4)))(encoder_embedding)
#         encoder_lstm2, forward_h2, forward_c2, backward_h2, backward_c2 = Bidirectional(LSTM(latent_dim, return_sequences=True, return_state=True,
#                                                             name='encoder_lstm_2', dropout=dropout_rate, recurrent_dropout=recurrent_dropout_rate,
#                                                                                              kernel_regularizer=l2(1e-4)))(encoder_lstm1)

#         state_h = Concatenate()([forward_h2, backward_h2])
#         state_c = Concatenate()([forward_c2, backward_c2])

#         # Decoder
#         decoder_inputs = Input(shape=(7, y_max_sequence_length,), name='decoder_inputs')
#         decoder_embedding = TimeDistributed(Embedding(input_dim=self.y_vocab_size_input, output_dim=self.embedding_dim, weights=[self.embedding_matrix_input],
#                                                       trainable=False))(decoder_inputs)

#         decoder_outputs = []
#         for i in range(7):
#             decoder_lstm_1 = LSTM(latent_dim * 2, return_sequences=True, return_state=False, name=f'decoder_lstm_1_{i}',
#                                   dropout=dropout_rate, recurrent_dropout=recurrent_dropout_rate, kernel_regularizer=l2(1e-4))
#             decoder_lstm_2 = LSTM(latent_dim * 2, return_sequences=True, return_state=False, name=f'decoder_lstm_2_{i}',
#                                   dropout=dropout_rate, recurrent_dropout=recurrent_dropout_rate, kernel_regularizer=l2(1e-4))
#             decoder_lstm_output_1 = decoder_lstm_1(decoder_embedding[:, i, :], initial_state=[state_h, state_c])
#             decoder_lstm_output_2 = decoder_lstm_2(decoder_lstm_output_1, initial_state=[state_h, state_c])
#             decoder_dense = TimeDistributed(Dense(self.y_vocab_size_input, activation='softmax'), name=f'decoder_dense_{i}')
#             decoder_outputs.append(decoder_dense(decoder_lstm_output_2))

#         optimizer = Adam(learning_rate=learning_rate)
#         model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
#         # model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#         model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#         return model

In [ ]:
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Concatenate, Dense, TimeDistributed
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.regularizers import l1_l2
# from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# from kerastuner import HyperModel, RandomSearch
# from keras_tuner import HyperModel
# import keras_tuner as kt


# class NLPHyperModel(HyperModel):
#     def __init__(self, vocab_size_input, embedding_dim, max_sequence_length, embedding_matrix_input, y_vocab_size_input, y_max_sequence_length):
#         self.vocab_size_input = vocab_size_input
#         self.embedding_dim = embedding_dim
#         self.max_sequence_length = max_sequence_length
#         self.embedding_matrix_input = embedding_matrix_input
#         self.y_vocab_size_input = y_vocab_size_input
#         self.y_max_sequence_length = y_max_sequence_length

#     def build(self, hp):
#         latent_dim = hp.Int('latent_dim', min_value=32, max_value=256, step=32)
#         dropout_rate = hp.Float('dropout_rate', min_value=0.2, max_value=0.7, step=0.1)
#         recurrent_dropout_rate = hp.Float('recurrent_dropout_rate', min_value=0.2, max_value=0.7, step=0.1)
#         learning_rate = hp.Float('learning_rate', min_value=0.001, max_value=0.01, sampling='LOG')
#         # l1_reg = hp.Float('l1_reg', min_value=1e-6, max_value=1e-2, sampling='LOG')
#         # l2_reg = hp.Float('l2_reg', min_value=1e-6, max_value=1e-2, sampling='LOG')
#         l1_reg = hp.Float('l1_reg', min_value=0.001, max_value=0.01, sampling='LOG')
#         l2_reg = hp.Float('l2_reg', min_value=0.001, max_value=0.01, sampling='LOG')

#         # Encoder
#         encoder_inputs = Input(shape=(self.max_sequence_length,), name='encoder_inputs')
#         encoder_embedding = Embedding(input_dim=self.vocab_size_input, output_dim=self.embedding_dim, weights=[self.embedding_matrix_input],
#                                       input_length=self.max_sequence_length, trainable=False, embeddings_regularizer=l1_l2(l1=l1_reg, l2=l2_reg))(encoder_inputs)
#         encoder_lstm1 = Bidirectional(LSTM(latent_dim, return_sequences=True, name='encoder_lstm_1', dropout=dropout_rate,
#                                            recurrent_dropout=recurrent_dropout_rate, kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg)))(encoder_embedding)
#         encoder_lstm2, forward_h2, forward_c2, backward_h2, backward_c2 = Bidirectional(LSTM(latent_dim, return_sequences=True, return_state=True,
#                                                             name='encoder_lstm_2', dropout=dropout_rate, recurrent_dropout=recurrent_dropout_rate, kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg)))(encoder_lstm1)

#         state_h = Concatenate()([forward_h2, backward_h2])
#         state_c = Concatenate()([forward_c2, backward_c2])

#         # Decoder
#         decoder_inputs = Input(shape=(31, self.y_max_sequence_length,), name='decoder_inputs')
#         decoder_embedding = TimeDistributed(Embedding(input_dim=self.y_vocab_size_input, output_dim=self.embedding_dim, weights=[self.embedding_matrix_input],
#                                                       trainable=False, embeddings_regularizer=l1_l2(l1=l1_reg, l2=l2_reg)))(decoder_inputs)

#         decoder_outputs = []
#         for i in range(31):
#             decoder_lstm_1 = LSTM(latent_dim * 2, return_sequences=True, return_state=False, name=f'decoder_lstm_1_{i}',
#                                   dropout=dropout_rate, recurrent_dropout=recurrent_dropout_rate, kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg))
#             decoder_lstm_2 = LSTM(latent_dim * 2, return_sequences=True, return_state=False, name=f'decoder_lstm_2_{i}',
#                                   dropout=dropout_rate, recurrent_dropout=recurrent_dropout_rate, kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg))
#             decoder_lstm_output_1 = decoder_lstm_1(decoder_embedding[:, i, :], initial_state=[state_h, state_c])
#             decoder_lstm_output_2 = decoder_lstm_2(decoder_lstm_output_1, initial_state=[state_h, state_c])
#             decoder_dense = TimeDistributed(Dense(self.y_vocab_size_input, activation='softmax', kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg)), name=f'decoder_dense_{i}')
#             decoder_outputs.append(decoder_dense(decoder_lstm_output_2))

#         model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
#         optimizer = Adam(learning_rate=learning_rate)
#         model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#         return model

In [ ]:
      ##### model with transformer encoder decoder

# import tensorflow as tf
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Embedding, Dense, LayerNormalization, Dropout, TimeDistributed, Add
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# from tensorflow.keras.regularizers import l1_l2
# from tensorflow.keras.layers import Layer
# from keras_tuner import HyperModel, RandomSearch

# class TransformerBlock(Layer):
#     def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
#         super(TransformerBlock, self).__init__()
#         self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
#         self.ffn = tf.keras.Sequential(
#             [Dense(ff_dim, activation="relu"), Dense(embed_dim),]
#         )
#         self.layernorm1 = LayerNormalization(epsilon=1e-6)
#         self.layernorm2 = LayerNormalization(epsilon=1e-6)
#         self.dropout1 = Dropout(rate)
#         self.dropout2 = Dropout(rate)

#     def call(self, inputs, training):
#         attn_output = self.att(inputs, inputs)
#         attn_output = self.dropout1(attn_output, training=training)
#         out1 = self.layernorm1(inputs + attn_output)
#         ffn_output = self.ffn(out1)
#         ffn_output = self.dropout2(ffn_output, training=training)
#         return self.layernorm2(out1 + ffn_output)

# class NLPHyperModel(HyperModel):
#     def __init__(self, vocab_size_input, embedding_dim, max_sequence_length, embedding_matrix_input, y_vocab_size_input, y_max_sequence_length):
#         self.vocab_size_input = vocab_size_input
#         self.embedding_dim = embedding_dim
#         self.max_sequence_length = max_sequence_length
#         self.embedding_matrix_input = embedding_matrix_input
#         self.y_vocab_size_input = y_vocab_size_input
#         self.y_max_sequence_length = y_max_sequence_length

#     def build(self, hp):
#         embed_dim = hp.Int('embed_dim', min_value=32, max_value=256, step=32)
#         num_heads = hp.Int('num_heads', min_value=2, max_value=8, step=2)
#         ff_dim = hp.Int('ff_dim', min_value=32, max_value=256, step=32)
#         dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)
#         learning_rate = hp.Float('learning_rate', min_value=0.001, max_value=0.01, sampling='LOG')
#         # l1_reg = hp.Float('l1_reg', min_value=1e-6, max_value=1e-2, sampling='LOG')
#         # l2_reg = hp.Float('l2_reg', min_value=1e-6, max_value=1e-2, sampling='LOG')
#         l1_reg = hp.Float('l1_reg', min_value=0.001, max_value=0.01, sampling='LOG')
#         l2_reg = hp.Float('l2_reg', min_value=0.001, max_value=0.01, sampling='LOG')


#         # Encoder
#         encoder_inputs = Input(shape=(self.max_sequence_length,), name='encoder_inputs')
#         encoder_embedding = Embedding(input_dim=self.vocab_size_input, output_dim=self.embedding_dim, weights=[self.embedding_matrix_input],
#                                       input_length=self.max_sequence_length, trainable=False, embeddings_regularizer=l1_l2(l1=l1_reg, l2=l2_reg))(encoder_inputs)
#         encoder_transformer_block = TransformerBlock(self.embedding_dim, num_heads, ff_dim, dropout_rate)
#         encoder_outputs = encoder_transformer_block(encoder_embedding)

#         # Decoder
#         decoder_inputs = Input(shape=(31, self.y_max_sequence_length,), name='decoder_inputs')
#         decoder_embedding = TimeDistributed(Embedding(input_dim=self.y_vocab_size_input, output_dim=self.embedding_dim, weights=[self.embedding_matrix_input],
#                                                       trainable=False, embeddings_regularizer=l1_l2(l1=l1_reg, l2=l2_reg)))(decoder_inputs)
#         decoder_transformer_blocks = [TransformerBlock(self.embedding_dim, num_heads, ff_dim, dropout_rate) for _ in range(31)]
#         decoder_outputs = [decoder_transformer_blocks[i](decoder_embedding[:, i, :]) for i in range(31)]
#         decoder_dense = TimeDistributed(Dense(self.y_vocab_size_input, activation='softmax', kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg)))
#         decoder_outputs = [decoder_dense(output) for output in decoder_outputs]

#         model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
#         optimizer = Adam(learning_rate=learning_rate)
#         model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#         return model

In [ ]:
# from keras.layers import Embedding, LSTM, Bidirectional, Concatenate, Dense, Input, TimeDistributed, LayerNormalization, Dropout
# from keras.models import Model
# from keras.regularizers import l1_l2
# from keras.optimizers import Adam
# from keras.callbacks import EarlyStopping, ReduceLROnPlateau
# import keras_tuner as kt
# from keras_tuner import HyperModel
# from tensorflow.keras.layers import Layer
# import tensorflow as tf

# class TransformerBlock(Layer):
#     def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
#         super(TransformerBlock, self).__init__()
#         self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
#         self.ffn = tf.keras.Sequential(
#             [Dense(ff_dim, activation="relu"), Dense(embed_dim),]
#         )
#         self.layernorm1 = LayerNormalization(epsilon=1e-5)
#         self.layernorm2 = LayerNormalization(epsilon=1e-5)
#         self.dropout1 = Dropout(rate)
#         self.dropout2 = Dropout(rate)

#     def call(self, inputs, training):
#         attn_output = self.att(inputs, inputs)
#         attn_output = self.dropout1(attn_output, training=training)
#         out1 = self.layernorm1(inputs + attn_output)
#         ffn_output = self.ffn(out1)
#         ffn_output = self.dropout2(ffn_output, training=training)
#         return self.layernorm2(out1 + ffn_output)


# class NLPHyperModel(HyperModel):
#     def __init__(self, vocab_size_input, embedding_dim, max_sequence_length, embedding_matrix_input, y_vocab_size_input, y_max_sequence_length):
#         self.vocab_size_input = vocab_size_input
#         self.embedding_dim = embedding_dim
#         self.max_sequence_length = max_sequence_length
#         self.embedding_matrix_input = embedding_matrix_input
#         self.y_vocab_size_input = y_vocab_size_input
#         self.y_max_sequence_length = y_max_sequence_length

#     def build(self, hp):
#         latent_dim = hp.Int('latent_dim', min_value=32, max_value=128, step=32)
#         dropout_rate = hp.Float('dropout_rate', min_value=0.2, max_value=0.7, step=0.1)
#         recurrent_dropout_rate = hp.Float('recurrent_dropout_rate', min_value=0.2, max_value=0.7, step=0.1)
#         num_heads = hp.Int('num_heads', min_value=2, max_value=8, step=2)
#         ff_dim = hp.Int('ff_dim', min_value=32, max_value=256, step=32)
#         l1_reg = hp.Float('l1', min_value=1e-5, max_value=1e-2, sampling='LOG')
#         l2_reg = hp.Float('l2', min_value=1e-5, max_value=1e-2, sampling='LOG')
#         learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
#         # learning_rate = hp.Float('learning_rate', min_value=0.0001, max_value=0.01, sampling='LOG')
#         # l1_reg = hp.Float('l1_reg', min_value=0.0001, max_value=0.01, sampling='LOG')
#         # l2_reg = hp.Float('l2_reg', min_value=0.0001, max_value=0.01, sampling='LOG')


#         # Encoder
#         encoder_inputs = Input(shape=(self.max_sequence_length,), name='encoder_inputs')
#         encoder_embedding = Embedding(
#             input_dim=self.vocab_size_input,
#             output_dim=self.embedding_dim,
#             weights=[self.embedding_matrix_input],
#             input_length=self.max_sequence_length,
#             trainable=False,
#             # embeddings_regularizer=l1_l2(l1=l1_reg, l2=l2_reg)
#             embeddings_regularizer=l2(1e-4)
#         )(encoder_inputs)

#         encoder_lstm1 = Bidirectional(LSTM(latent_dim, return_sequences=True, name='encoder_lstm_1',
#                                            dropout=dropout_rate, recurrent_dropout=recurrent_dropout_rate, kernel_regularizer=l2(1e-4),
#                                            recurrent_regularizer=l2(1e-4)))(encoder_embedding)
#         encoder_lstm2, forward_h2, forward_c2, backward_h2, backward_c2 = Bidirectional(LSTM(latent_dim, return_sequences=True,
#                                                                                              return_state=True, name='encoder_lstm_2',
#                                                                                              dropout=dropout_rate,
#                                                                                              recurrent_dropout=recurrent_dropout_rate,
#                                                                                              kernel_regularizer=l2(1e-4),
#                                                                                              recurrent_regularizer=l2(1e-4)))(encoder_lstm1)

#         encoder_transformer_block = TransformerBlock(latent_dim * 2, num_heads, ff_dim, dropout_rate)
#         encoder_outputs = encoder_transformer_block(encoder_lstm2)

#         state_h = Concatenate()([forward_h2, backward_h2])  # forward_h, backward_h
#         state_c = Concatenate()([forward_c2, backward_c2])  # forward_c, backward_c

#         # Decoder
#         decoder_inputs = Input(shape=(7, self.y_max_sequence_length,), name='decoder_inputs')
#         decoder_embedding = TimeDistributed(Embedding(
#             input_dim=self.y_vocab_size_input,
#             output_dim=self.embedding_dim,
#             weights=[self.embedding_matrix_input],
#             trainable=False,
#             # embeddings_regularizer=l1_l2(l1=l1_reg, l2=l2_reg)
#             embeddings_regularizer=l2(1e-4)
#         ))(decoder_inputs)

#         decoder_outputs = []
#         for i in range(7):
#             decoder_lstm_1 = LSTM(latent_dim * 2, return_sequences=True, return_state=False, name=f'decoder_lstm_1_{i}', dropout=dropout_rate,
#                                   recurrent_dropout=recurrent_dropout_rate, kernel_regularizer=l2(1e-4), recurrent_regularizer=l2(1e-4))
#             decoder_lstm_2 = LSTM(latent_dim * 2, return_sequences=True, return_state=False, name=f'decoder_lstm_2_{i}', dropout=dropout_rate,
#                                   recurrent_dropout=recurrent_dropout_rate, kernel_regularizer=l2(1e-4), recurrent_regularizer=l2(1e-4))
#             decoder_lstm_output_1 = decoder_lstm_1(decoder_embedding[:, i, :], initial_state=[state_h, state_c])
#             decoder_lstm_output_2 = decoder_lstm_2(decoder_lstm_output_1, initial_state=[state_h, state_c])
#             # decoder_dense = TimeDistributed(Dense(self.y_vocab_size_input, activation='softmax', kernel_regularizer=l2(1e-4)),
#             #                                 name=f'decoder_dense_{i}')
#             decoder_dense = TimeDistributed(Dense(self.y_vocab_size_input, activation='softmax', kernel_regularizer=l2(1e-4)),
#                                             name=f'decoder_dense_{i}')
#             decoder_outputs.append(decoder_dense(decoder_lstm_output_2))

#         model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
#         optimizer = Adam(learning_rate=learning_rate)
#         model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#         return model


In [ ]:
# from keras_tuner import RandomSearch

# # Define the hypermodel
# hypermodel = NLPHyperModel(
#     vocab_size_input=vocab_size_input,
#     embedding_dim=embedding_dim,
#     max_sequence_length=max_sequence_length,
#     embedding_matrix_input=embedding_matrix_input,
#     y_vocab_size_input=y_vocab_size_input,
#     y_max_sequence_length= y_max_sequence_length
# )

# # Initialize the tuner
# tuner = RandomSearch(
#     hypermodel,
#     objective='val_loss',
#     max_trials=1,   ## number of model it will search
#     executions_per_trial=1,   ## numberof time each model will train
#     directory='my_dir',
#     project_name='nlp_tuning'
# )

# from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# # Define the EarlyStopping callback
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-5, cooldown=0)

# # Run the tuner search
# # tuner.search([X_train, y_train], [y_train[:, i, :] for i in range(y_train.shape[1])], epochs=50,
# #              validation_split=0.2, callbacks=[early_stopping, reduce_lr])
# tuner.search([X_train, y_train], [y_train[:, i, :] for i in range(y_train.shape[1])], epochs=30, batch_size= 64,
#              validation_split=0.2, callbacks=[early_stopping, reduce_lr])


In [ ]:
# best_model = tuner.get_best_models(num_models=1)[0]
# best_model.summary()

In [ ]:
# # Define the EarlyStopping callback
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-5, cooldown=0)

# # Train the best model further if needed
# history = best_model.fit([X_train, y_train], [y_train[:, i, :] for i in range(y_train.shape[1])], epochs=50, validation_split=0.2, batch_size= 64,
#                          callbacks=[reduce_lr, early_stopping])


In [ ]:
from keras.layers import Attention

# Encoder
encoder_inputs = Input(shape=(max_sequence_length,), name='encoder_inputs')
encoder_embedding = Embedding(input_dim=vocab_size_input, output_dim=embedding_dim, weights=[embedding_matrix_input],
                              input_length=max_sequence_length, trainable=False)(encoder_inputs)
encoder_lstm1 = Bidirectional(LSTM(96, return_sequences=True, name='encoder_lstm_1', dropout=0.3,
                                    recurrent_dropout=0.3, kernel_regularizer=l2(1e-4)))(encoder_embedding)
encoder_lstm2, forward_h2, forward_c2, backward_h2, backward_c2 = Bidirectional(LSTM(96, return_sequences=True, return_state=True,
                                                    name='encoder_lstm_2', dropout=0.3, recurrent_dropout=0.3,
                                                    kernel_regularizer=l2(1e-4)))(encoder_lstm1)

state_h = Concatenate()([forward_h2, backward_h2])
state_c = Concatenate()([forward_c2, backward_c2])

# Decoder
decoder_inputs = Input(shape=(21, y_max_sequence_length,), name='decoder_inputs')
decoder_embedding = TimeDistributed(Embedding(input_dim=y_vocab_size_input, output_dim=embedding_dim, weights=[embedding_matrix_input],
                                              trainable=False))(decoder_inputs)

attention = Attention()

decoder_outputs = []
for i in range(21):
    decoder_lstm_1 = LSTM(96 * 2, return_sequences=True, return_state=True, name=f'decoder_lstm_1_{i}',
                          dropout=0.3, recurrent_dropout=0.3, kernel_regularizer=l2(1e-4))
    decoder_lstm_2 = LSTM(96 * 2, return_sequences=True, return_state=True, name=f'decoder_lstm_2_{i}',
                          dropout=0.3, recurrent_dropout=0.3, kernel_regularizer=l2(1e-4))
    
    lstm_1_output, _, _ = decoder_lstm_1(decoder_embedding[:, i, :], initial_state=[state_h, state_c])
    attention_output = attention([lstm_1_output, encoder_lstm2])
    decoder_lstm_output_2, _, _ = decoder_lstm_2(attention_output, initial_state=[state_h, state_c])
    
    decoder_dense = TimeDistributed(Dense(y_vocab_size_input, activation='softmax'), name=f'decoder_dense_{i}')
    decoder_outputs.append(decoder_dense(decoder_lstm_output_2))

optimizer = Adam(learning_rate=0.004872)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-5, cooldown=0)

model.summary()


In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau


encoder_inputs = Input(shape=(max_sequence_length,), name='encoder_inputs')
encoder_embedding = Embedding(input_dim=vocab_size_input, output_dim=embedding_dim, weights=[embedding_matrix_input],
                              input_length=max_sequence_length, trainable=False)(encoder_inputs)
encoder_lstm1 = Bidirectional(LSTM(96, return_sequences=True, name='encoder_lstm_1', dropout=0.3,
                                    recurrent_dropout=0.3, kernel_regularizer=l2(1e-4)))(encoder_embedding)
encoder_lstm2, forward_h2, forward_c2, backward_h2, backward_c2 = Bidirectional(LSTM(96, return_sequences=True, return_state=True,
                                                    name='encoder_lstm_2', dropout=0.3, recurrent_dropout=0.3,
                                                                                      kernel_regularizer=l2(1e-4)))(encoder_lstm1)

state_h = Concatenate()([forward_h2, backward_h2])
state_c = Concatenate()([forward_c2, backward_c2])

### best LSTM = 128

# Decoder
decoder_inputs = Input(shape=(21, y_max_sequence_length,), name='decoder_inputs')
decoder_embedding = TimeDistributed(Embedding(input_dim=y_vocab_size_input, output_dim=embedding_dim, weights=[embedding_matrix_input],
                                              trainable=False))(decoder_inputs)

decoder_outputs = []
for i in range(21):
    decoder_lstm_1 = LSTM(96 * 2, return_sequences=True, return_state=False, name=f'decoder_lstm_1_{i}',
                          dropout=0.3, recurrent_dropout=0.3, kernel_regularizer=l2(1e-4))
    decoder_lstm_2 = LSTM(96 * 2, return_sequences=True, return_state=False, name=f'decoder_lstm_2_{i}',
                          dropout=0.3, recurrent_dropout=0.3, kernel_regularizer=l2(1e-4))
    decoder_lstm_output_1 = decoder_lstm_1(decoder_embedding[:, i, :], initial_state=[state_h, state_c])
    decoder_lstm_output_2 = decoder_lstm_2(decoder_lstm_output_1, initial_state=[state_h, state_c])
    decoder_dense = TimeDistributed(Dense(y_vocab_size_input, activation='softmax'), name=f'decoder_dense_{i}')
    decoder_outputs.append(decoder_dense(decoder_lstm_output_2))

optimizer = Adam(learning_rate=0.004872)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-5, cooldown=0)

model.summary()

In [ ]:
model.fit([X_train, y_train], [y_train[:, i, :] for i in range(y_train.shape[1])], epochs=20, batch_size= 32,
             validation_split=0.2, callbacks=[early_stopping, reduce_lr])

Epoch 1/20
40/40 [==============================] - 1304s 29s/step - loss: 18.9756 - decoder_dense_0_loss: 1.4956 - decoder_dense_1_loss: 0.7833 - decoder_dense_2_loss: 0.9041 - decoder_dense_3_loss: 0.8109 - decoder_dense_4_loss: 0.8152 - decoder_dense_5_loss: 0.7785 - decoder_dense_6_loss: 0.8966 - decoder_dense_7_loss: 0.7591 - decoder_dense_8_loss: 0.8156 - decoder_dense_9_loss: 0.8137 - decoder_dense_10_loss: 0.9090 - decoder_dense_11_loss: 0.8018 - decoder_dense_12_loss: 0.8021 - decoder_dense_13_loss: 0.8509 - decoder_dense_14_loss: 0.7795 - decoder_dense_15_loss: 0.8098 - decoder_dense_16_loss: 0.8059 - decoder_dense_17_loss: 0.7751 - decoder_dense_18_loss: 0.8275 - decoder_dense_19_loss: 1.0161 - decoder_dense_20_loss: 0.9051 - decoder_dense_0_accuracy: 0.8547 - decoder_dense_1_accuracy: 0.9643 - decoder_dense_2_accuracy: 0.9276 - decoder_dense_3_accuracy: 0.9503 - decoder_dense_4_accuracy: 0.9566 - decoder_dense_5_accuracy: 0.9631 - decoder_dense_6_accuracy: 0.9317 - decoder_

In [ ]:
# model.save('model.h5')

model.save('my_model.keras')

In [ ]:
input_sentence = "Ho flatulence since one year . Abdomen problem . Complains from more than 1 year ago . Gradually it happen . No attack . Patient does not have smoking habit . Does not drink alcohol . "
input_sequence = tokenizer.texts_to_sequences([input_sentence])
input_sequence = pad_sequences(input_sequence, maxlen=max_sequence_length, padding='post')

decoder_input_sequence = np.zeros((1, 21, y_max_sequence_length))

predictions = model.predict([input_sequence, decoder_input_sequence])

predicted_sentences = []
for i in range(21):
    predicted_indices = np.argmax(predictions[i], axis=-1)
    predicted_sentence = ' '.join(tokenizer.index_word.get(idx, '') for idx in predicted_indices[0] if idx > 0)
    predicted_sentences.append(predicted_sentence)

for i, sentence in enumerate(predicted_sentences):
    print(f"Predicted sentence for column {i+1}: {sentence}")

In [73]:
# model.summary()

In [74]:
from tensorflow.keras.models import load_model

pretrained_model = load_model('/content/my_model.keras')

In [75]:
pretrained_model.fit([X_train, y_train], [y_train[:, i, :] for i in range(y_train.shape[1])], epochs=20, batch_size= 32,
             validation_split=0.2, callbacks=[early_stopping, reduce_lr])

Epoch 1/20
40/40 [==============================] - 1348s 31s/step - loss: 0.6308 - decoder_dense_0_loss: 0.3469 - decoder_dense_1_loss: 0.0032 - decoder_dense_2_loss: 0.0164 - decoder_dense_3_loss: 0.0040 - decoder_dense_4_loss: 0.0095 - decoder_dense_5_loss: 0.0020 - decoder_dense_6_loss: 0.0055 - decoder_dense_7_loss: 0.0043 - decoder_dense_8_loss: 0.0057 - decoder_dense_9_loss: 0.0122 - decoder_dense_10_loss: 0.0032 - decoder_dense_11_loss: 0.0036 - decoder_dense_12_loss: 0.0014 - decoder_dense_13_loss: 0.0194 - decoder_dense_14_loss: 0.0036 - decoder_dense_15_loss: 0.0079 - decoder_dense_16_loss: 0.0042 - decoder_dense_17_loss: 0.0040 - decoder_dense_18_loss: 0.0028 - decoder_dense_19_loss: 0.0919 - decoder_dense_20_loss: 0.0170 - decoder_dense_0_accuracy: 0.9220 - decoder_dense_1_accuracy: 0.9991 - decoder_dense_2_accuracy: 0.9942 - decoder_dense_3_accuracy: 0.9986 - decoder_dense_4_accuracy: 0.9969 - decoder_dense_5_accuracy: 0.9996 - decoder_dense_6_accuracy: 0.9990 - decoder_d

In [76]:
pretrained_model.save('my_model.keras')

In [85]:
input_sentence_list = ["Ho intermittent pain in epigastric region since one year. It was associated with vomiting  flatulence . Abdomen problem . Complains from less than 1 year ago . Gradually it happen . Moderate problem . No attack . Patient does not have smoking habit . Light Worker . Does not drink alcohol . Non Vegetarian . Prefer meals 2 3 times a day . Prefer oil mustrad oil . Past medical history Surgery uterus tumour in 1991 . Past medical history Hypertensive  DM II on treatment . Medicine refered by other TAB. AMLOZ INJ. INSULIN . Acute resolving calculus cholecystitis Gangreenous with empyema . Treatment type OPERATIVE . Surgery type CHOLECYSTECTOMY . Procedure follow CHOLECYSTECTOMY   LAP . Anesthic type General . Anes smooth recovery . Recommende drug INJ. PYROLATE 1 AMP INJ. PROPOFOL .  INJ. ROCURONIUM . Findings  GB was grossly distended filled with pus Friable hartman s pouch Empyema GB Moderate large size calculi present Calot s area grossly edematous CBD normal . NORMAL recovery . Medicines suggested Inj.Tazar 4.5 gmInj.Ketanav 2 AMPInj.Pantocid 40 mg . Follow-up date 21 March 2013 . CHECK UP WITH BIOPSY REPORT AFTER 7 DAYS . ",
                  "loose stool mixed with blood melaena since one month. irregular bowel. sometimes he passed stool thrice daily. ho anaemia, acute gastritis  uti one month ago. open gb 1994 . Complains from 1 month ago . Gradually it happen . Moderate problem . No attack . Patient does not have smoking habit . Moderate Worker . Does not drink alcohol . Both VegNon Veg . Prefer meals 2 3 times a day . Prefer oil mustrad oil . WELL DIFFERENTIAT ADENOCARCINOMA RECTUM UPPER 3RD . Treatment type OPERATIVE . Surgery type MULTIPLE . Procedure follow lap assested anterior resection  stappled anastomosis . Anesthic type General . Anes smooth recovery . Recommende drug INJ. PYROLATE 1 AMP .  INJ. PROPOFOL . NORMAL recovery . Medicines suggested Normal 0 false false false EN IN X NONE X NONE MicrosoftInternetExplorer4 . Follow-up date 03 July 2013 . CHECK UP AFTER 7 DAYS . "]
for input_sentence in input_sentence_list:
  input_sequence = tokenizer.texts_to_sequences([input_sentence])
  input_sequence = pad_sequences(input_sequence, maxlen=max_sequence_length, padding='post')

  decoder_input_sequence = np.zeros((1, 21, y_max_sequence_length))

  predictions = pretrained_model.predict([input_sequence, decoder_input_sequence])

  predicted_sentences = []
  for i in range(21):
      predicted_indices = np.argmax(predictions[i], axis=-1)
      predicted_sentence = ' '.join(tokenizer.index_word.get(idx, '') for idx in predicted_indices[0] if idx > 0)
      predicted_sentences.append(predicted_sentence)

  for i, sentence in enumerate(predicted_sentences):
      print(f"Predicted sentence for column {i+1}: {sentence}")

1/1 [==============================] - 3s 3s/step
Predicted sentence for column 1: 1 report surgery amp medicines the history history acute medical medical mild mild findings findings
Predicted sentence for column 2: no inj
Predicted sentence for column 3: prefer procedure calculus propofol in meals operative
Predicted sentence for column 4: after medical ho
Predicted sentence for column 5: 7 inj
Predicted sentence for column 6: cholecystectomy up
Predicted sentence for column 7: 3 2 with months days happen
Predicted sentence for column 8: alcohol smoking
Predicted sentence for column 9: 2 with anes anesthic
Predicted sentence for column 10: recommende cholelithiasis
Predicted sentence for column 11: oil worker date past check drink mustrad
Predicted sentence for column 12: oil treatment smooth treatment
Predicted sentence for column 13: recovery follow does
Predicted sentence for column 14: attack follow ago
Predicted sentence for column 15: type
Predicted sentence for column 16: type

In [ ]:
# from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-5)
# # Fit the model
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# history = model.fit([X_train, y_train],  # Inputs are a list: [encoder_inputs, decoder_inputs]
#             [y_train[:, i, :] for i in range(y_train.shape[1])],  # Outputs are a list of decoder outputs
#             epochs=50,      ## 150 + 40 + 20 + 20
#             batch_size=16,
#             callbacks=[early_stopping, reduce_lr],
#             validation_data=([X_test, y_test],  # Validation inputs
#                             [y_test[:, i, :] for i in range(y_test.shape[1])]))  # Validation outputs

In [ ]:
# # Assuming evaluation_inputs and y_test are correctly formatted
# evaluation_inputs = [X_test, y_test]

# # Evaluate the model
# losses = model.evaluate(evaluation_inputs,  # Input should match [encoder_inputs, decoder_inputs]
#                         [y_test[:, i, :] for i in range(y_test.shape[1])],  # Output should match decoder outputs
#                         batch_size=16)

# # Print evaluation results
# for i, metric_name in enumerate(model.metrics_names):
#     print(f'Test {metric_name}: {losses[i]}')


In [ ]:
# # Example input sentence and preprocessing
# input_sentence = "Sophia Martinez, 38-year-old female nurse, visits on July 5, 2024. Complains of chronic back pain and fatigue. Describes it as a dull ache in the lower back. History of anemia. No surgical history. Father had osteoporosis. Advise DEXA scan and blood tests. Prescribe iron supplements and recommend physical therapy. Follow-up appointment in six weeks."
# input_sequence = tokenizer.texts_to_sequences([input_sentence])
# input_sequence = pad_sequences(input_sequence, maxlen=max_sequence_length, padding='post')

# # Prepare decoder input sequence
# decoder_input_sequence = np.zeros((1, 31, y_max_sequence_length))  # Assuming y_max_sequence_length is 200

# # Predict using the model
# predictions = model.predict([input_sequence, decoder_input_sequence])

# # Process predictions
# predicted_sentences = []
# for i in range(31):
#     predicted_indices = np.argmax(predictions[i], axis=-1)
#     predicted_sentence = ' '.join(tokenizer.index_word.get(idx, '') for idx in predicted_indices[0] if idx > 0)
#     predicted_sentences.append(predicted_sentence)

# # Print predicted sentences
# for i, sentence in enumerate(predicted_sentences):
#     print(f"Predicted sentence for column {i+1}: {sentence}")

In [ ]:
# encoder_model = Model(encoder_inputs, [state_h, state_c])
# decoder_state_input_h = Input(shape=(latent_dim * 2,), name='decoder_state_input_h')
# decoder_state_input_c = Input(shape=(latent_dim * 2,), name='decoder_state_input_c')

# decoder_inputs_single = Input(shape=(1,), name='decoder_inputs_single')
# decoder_embedding_single = Embedding(input_dim=vocab_size_input, output_dim=embedding_dim, weights=[embedding_matrix_input], trainable=False)(decoder_inputs_single)

# decoder_lstm_output, decoder_state_h, decoder_state_c = LSTM(latent_dim * 2, return_sequences=True, return_state=True, dropout=0.5, recurrent_dropout=0.5)(
#     decoder_embedding_single, initial_state=[decoder_state_input_h, decoder_state_input_c])

# decoder_dense_single = Dense(vocab_size_input, activation='softmax')
# decoder_outputs_single = decoder_dense_single(decoder_lstm_output)

# decoder_model = Model(
#     [decoder_inputs_single, decoder_state_input_h, decoder_state_input_c],
#     [decoder_outputs_single, decoder_state_h, decoder_state_c])


In [ ]:
# tokenizer.index_word

In [ ]:
# def predict_all_sentences(input_seq):
#     states_value = encoder_model.predict(input_seq)
#     all_decoded_sentences = []

#     for i in range(31):
#         decoded_sentence = ''
#         target_seq = np.zeros((1, 1))
#         target_seq[0, 0] = start_token_id
#         stop_condition = False

#         while not stop_condition:
#             output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
#             sampled_token_index = np.argmax(output_tokens[0, -1, :])
#             sampled_char = tokenizer.index_word[sampled_token_index]
#             decoded_sentence += ' ' + sampled_char

#             # if (sampled_char == 'END' or len(decoded_sentence) > 200):
#             if (sampled_char == '<end>' or len(decoded_sentence) > 200):
#                 stop_condition = True

#             target_seq = np.zeros((1, 1))
#             target_seq[0, 0] = sampled_token_index
#             states_value = [h, c]

#         all_decoded_sentences.append(decoded_sentence)

#     return all_decoded_sentences

# # Example usage:
# input_seq = X_test[0:1]  # Assuming X_test is prepared similar to X_train
# predicted_sentences = predict_all_sentences(input_seq)
# print(predicted_sentences)


In [ ]:
# X_test.shape

In [ ]:
# predictions[0][0]